In [1]:
! pip install bigartm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.8 MB/s 


In [2]:
!pip install --upgrade gensim==3.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [5]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this 

In [7]:
!ls

esf-for-tm-sample1mln.zip  mallet-2.0.8  mallet-2.0.8.zip  sample_data


In [ ]:
from pprint import pprint
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models.wrappers import LdaMallet

In [9]:
%%time
res = pd.read_csv('esf-for-tm-sample1mln.zip')
#res = res.sample(100_000)
data_ready = res.DESCRIPTION.str.split().tolist()

CPU times: user 2.39 s, sys: 165 ms, total: 2.55 s
Wall time: 2.57 s


In [12]:
%%time
id2word = corpora.Dictionary(data_ready)
print('Total Vocabulary Size:', len(id2word))
corpus = [id2word.doc2bow(text) for text in data_ready]

Total Vocabulary Size: 252025
CPU times: user 30.1 s, sys: 444 ms, total: 30.6 s
Wall time: 36.7 s


In [14]:
%%time
ldamallet = gensim.models.wrappers.ldamallet.LdaMallet(mallet_path,
                                                       corpus=corpus,
                                                       num_topics=1000,
                                                       id2word=id2word)

CPU times: user 1min 11s, sys: 5.26 s, total: 1min 16s
Wall time: 2h 3min 23s


In [15]:
topics = ldamallet.show_topics(num_topics = 500, formatted=False)
dff = list()
for i in topics:
    dff.append({'num_topic':i[0],
               'tokens':', '.join(j[0] for j in i[1])})
dff = pd.DataFrame(dff)
dff.to_excel('1000topics-from-1mln.xlsx', index=False)

In [16]:
ldamallet.save( 'lda-1000-1mln.pkl')

In [ ]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/